# Modelo de Regressão Linear - Predição de Preços de Imóveis Rurais

**Autor:** Marcos Paulo Roriz Lima Reis  
**RA:** 22007534  
**Curso:** Engenharia da Computação - UniCEUB

## Objetivo
Desenvolver e avaliar um modelo de regressão linear para predição de preços de imóveis rurais baseado em suas características.

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Bibliotecas de Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Seed para reprodutibilidade
np.random.seed(42)

## 2. Carregamento dos Dados

In [ ]:
# Carregar dados processados
data_path = Path('../data/processed/imoveis_processados.csv')

if data_path.exists():
    df = pd.read_csv(data_path)
    print(f"Dados carregados com sucesso! Shape: {df.shape}")
    print("\nPrimeiras linhas:")
    display(df.head())
else:
    print("Arquivo de dados processados não encontrado.")
    print("Execute o notebook 'eda.ipynb' primeiro para processar os dados.")
    df = pd.DataFrame()

## 3. Preparação dos Dados para Modelagem

In [ ]:
# Verificar dados disponíveis
print("Colunas disponíveis:")
print(df.columns.tolist())

print("\nInformações do dataset:")
df.info()

In [ ]:
# Selecionar features (variáveis independentes) e target (variável dependente)
# Para este exemplo, vamos usar 'area' como feature principal
# e 'preco' como target

if 'preco' in df.columns and 'area' in df.columns:
    # Remover valores nulos
    df_model = df[['area', 'preco']].dropna()
    
    # Separar features e target
    X = df_model[['area']].values
    y = df_model['preco'].values
    
    print(f"Shape de X (features): {X.shape}")
    print(f"Shape de y (target): {y.shape}")
    print(f"\nNúmero de amostras para treinamento: {len(X)}")
else:
    print("Colunas 'preco' ou 'area' não encontradas no dataset.")

## 4. Divisão dos Dados (Train/Test Split)

In [ ]:
# Dividir dados em conjunto de treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Divisão dos dados:")
print(f"  - Treino: {len(X_train)} amostras ({len(X_train)/len(X)*100:.1f}%)")
print(f"  - Teste:  {len(X_test)} amostras ({len(X_test)/len(X)*100:.1f}%)")

## 5. Treinamento do Modelo de Regressão Linear

In [ ]:
# Criar e treinar o modelo
model = LinearRegression()
model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")
print(f"\nParâmetros do modelo:")
print(f"  - Coeficiente (slope): R$ {model.coef_[0]:,.2f} por hectare")
print(f"  - Intercepto: R$ {model.intercept_:,.2f}")

# Equação da regressão
print(f"\nEquação da Regressão Linear:")
print(f"Preço = {model.intercept_:,.2f} + {model.coef_[0]:,.2f} × Área")

## 6. Predições e Avaliação do Modelo

In [ ]:
# Fazer predições no conjunto de treino
y_train_pred = model.predict(X_train)

# Fazer predições no conjunto de teste
y_test_pred = model.predict(X_test)

print("Predições realizadas com sucesso!")

In [ ]:
# Calcular métricas de avaliação
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Calcula e exibe métricas de avaliação do modelo."""
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    print(f"\nMétricas de Avaliação - {dataset_name}:")
    print(f"  - R² (Coeficiente de Determinação): {r2:.4f}")
    print(f"  - MSE (Mean Squared Error): R$ {mse:,.2f}")
    print(f"  - RMSE (Root Mean Squared Error): R$ {rmse:,.2f}")
    print(f"  - MAE (Mean Absolute Error): R$ {mae:,.2f}")
    
    return {'r2': r2, 'mse': mse, 'rmse': rmse, 'mae': mae}

# Avaliar no conjunto de treino
train_metrics = evaluate_model(y_train, y_train_pred, "Conjunto de Treino")

# Avaliar no conjunto de teste
test_metrics = evaluate_model(y_test, y_test_pred, "Conjunto de Teste")

### Interpretação das Métricas:

- **R² (R-squared)**: Indica quanto da variabilidade do preço é explicada pela área. Valores próximos de 1 indicam melhor ajuste.
- **RMSE (Root Mean Squared Error)**: Erro médio das predições em reais. Quanto menor, melhor.
- **MAE (Mean Absolute Error)**: Erro absoluto médio. Mais robusto a outliers que o RMSE.

## 7. Validação Cruzada

In [ ]:
# Realizar validação cruzada (5-fold)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

print("Resultados da Validação Cruzada (5-fold):")
print(f"  - R² scores: {cv_scores}")
print(f"  - R² médio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 8. Visualizações

In [ ]:
# Visualização: Dados reais vs. Linha de regressão
plt.figure(figsize=(12, 6))

# Plotar dados de treino
plt.scatter(X_train, y_train, alpha=0.5, label='Dados de Treino', color='blue')

# Plotar dados de teste
plt.scatter(X_test, y_test, alpha=0.5, label='Dados de Teste', color='green')

# Plotar linha de regressão
X_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_range_pred = model.predict(X_range)
plt.plot(X_range, y_range_pred, color='red', linewidth=2, label='Linha de Regressão')

plt.xlabel('Área (hectares)', fontsize=12)
plt.ylabel('Preço (R$)', fontsize=12)
plt.title('Regressão Linear: Área vs. Preço', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualização: Valores Reais vs. Preditos
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Conjunto de Treino
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([y_train.min(), y_train.max()], 
             [y_train.min(), y_train.max()], 
             'r--', lw=2, label='Predição Perfeita')
axes[0].set_xlabel('Preço Real (R$)', fontsize=12)
axes[0].set_ylabel('Preço Predito (R$)', fontsize=12)
axes[0].set_title('Conjunto de Treino', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Conjunto de Teste
axes[1].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[1].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Predição Perfeita')
axes[1].set_xlabel('Preço Real (R$)', fontsize=12)
axes[1].set_ylabel('Preço Predito (R$)', fontsize=12)
axes[1].set_title('Conjunto de Teste', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análise de Resíduos
residuals_train = y_train - y_train_pred
residuals_test = y_test - y_test_pred

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Resíduos vs. Valores Preditos (Treino)
axes[0, 0].scatter(y_train_pred, residuals_train, alpha=0.5, color='blue')
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Valores Preditos', fontsize=11)
axes[0, 0].set_ylabel('Resíduos', fontsize=11)
axes[0, 0].set_title('Resíduos - Conjunto de Treino', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Resíduos vs. Valores Preditos (Teste)
axes[0, 1].scatter(y_test_pred, residuals_test, alpha=0.5, color='green')
axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Valores Preditos', fontsize=11)
axes[0, 1].set_ylabel('Resíduos', fontsize=11)
axes[0, 1].set_title('Resíduos - Conjunto de Teste', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Histograma de Resíduos (Treino)
axes[1, 0].hist(residuals_train, bins=30, edgecolor='black', color='blue', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Resíduos', fontsize=11)
axes[1, 0].set_ylabel('Frequência', fontsize=11)
axes[1, 0].set_title('Distribuição dos Resíduos - Treino', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Histograma de Resíduos (Teste)
axes[1, 1].hist(residuals_test, bins=30, edgecolor='black', color='green', alpha=0.7)
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Resíduos', fontsize=11)
axes[1, 1].set_ylabel('Frequência', fontsize=11)
axes[1, 1].set_title('Distribuição dos Resíduos - Teste', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Fazendo Novas Predições

In [ ]:
# Exemplo de predição para novos imóveis
def predict_price(area_hectares):
    """Prediz o preço de um imóvel baseado em sua área."""
    area_array = np.array([[area_hectares]])
    predicted_price = model.predict(area_array)[0]
    return predicted_price

# Exemplos de predição
test_areas = [10, 50, 100, 200, 500]

print("\nExemplos de Predições:")
print("=" * 50)
for area in test_areas:
    price = predict_price(area)
    print(f"Área: {area:6.1f} hectares → Preço previsto: R$ {price:12,.2f}")

## 10. Salvamento do Modelo

In [ ]:
# Salvar o modelo treinado
import pickle

model_path = Path('../data/processed')
model_path.mkdir(parents=True, exist_ok=True)

# Salvar modelo
model_file = model_path / 'linear_regression_model.pkl'
with open(model_file, 'wb') as f:
    pickle.dump(model, f)

print(f"Modelo salvo em: {model_file}")

# Salvar métricas
metrics_df = pd.DataFrame({
    'Dataset': ['Treino', 'Teste'],
    'R2': [train_metrics['r2'], test_metrics['r2']],
    'RMSE': [train_metrics['rmse'], test_metrics['rmse']],
    'MAE': [train_metrics['mae'], test_metrics['mae']]
})

metrics_file = model_path / 'model_metrics.csv'
metrics_df.to_csv(metrics_file, index=False)
print(f"Métricas salvas em: {metrics_file}")

## 11. Conclusões

### Resultados do Modelo:

1. **Performance do Modelo:**
   - O modelo de regressão linear foi treinado com sucesso
   - Avaliar o R² para entender a capacidade preditiva
   - RMSE e MAE indicam o erro médio das predições

2. **Análise de Resíduos:**
   - Verificar se os resíduos estão distribuídos aleatoriamente
   - Resíduos com padrões indicam possível necessidade de modelos mais complexos

3. **Limitações:**
   - Modelo simples usando apenas a área como feature
   - Pode ser melhorado com mais variáveis (localização, infraestrutura, etc.)
   - Relação linear pode não capturar toda a complexidade do mercado

4. **Aplicações:**
   - Estimativa rápida de preços para novos imóveis
   - Base para desenvolvimento de modelos mais sofisticados
   - Ferramenta de apoio à tomada de decisão

### Melhorias Futuras:
- Incluir mais features (localização, infraestrutura, tipo de solo)
- Testar outros algoritmos (Ridge, Lasso, Random Forest)
- Realizar feature engineering mais avançado
- Coletar mais dados para melhorar a generalização

---

**Projeto desenvolvido por Marcos Paulo Roriz Lima Reis (RA: 22007534)**  
**Engenharia da Computação - UniCEUB**